In [5]:
import openai, json

client = openai.OpenAI()
messages = []

In [6]:
def get_weather(city):
    return "33 degrees celcius."


FUNCTION_MAP = {"get_weather": get_weather}

In [7]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [8]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

In [9]:
def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [10]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: my name is jun
AI: Hello, Jun! How can I assist you today?
User: how is the weather of seoul?
Calling function: get_weather with {"city":"Seoul"}
User: go
AI: The weather in Seoul is currently 33 degrees Celsius. If you need any more information or assistance, feel free to ask!


In [11]:
messages

[{'role': 'user', 'content': 'my name is jun'},
 {'role': 'assistant', 'content': 'Hello, Jun! How can I assist you today?'},
 {'role': 'user', 'content': 'how is the weather of seoul?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_edHdOOtHHRjh1vpBkNEvLiqa',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_edHdOOtHHRjh1vpBkNEvLiqa',
  'name': 'get_weather',
  'content': '33 degrees celcius.'},
 {'role': 'user', 'content': 'go'},
 {'role': 'assistant',
  'content': 'The weather in Seoul is currently 33 degrees Celsius. If you need any more information or assistance, feel free to ask!'}]